In [32]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import myslack_incomming
import scipy
import pickle
from sklearn.ensemble import GradientBoostingRegressor 

## Preprocessing Step2 -  Encoding

In [2]:
myslack_incomming.send_slack('Encoding started.')

<Response [200]>


In [3]:
train = pickle.load(open("./data/imputation_train.pkl", "rb"))
test = pickle.load(open("./data/imputation_test.pkl", "rb"))

In [4]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,3600.0


---

In [90]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,1,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,1,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,1,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,1,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,1,3.800060e+09,1,GROCERY DRY GOODS,3600.0


In [100]:
train[train['VisitNumber']==8].groupby('DepartmentDescription')['ScanCount'].sum()

DepartmentDescription
DAIRY                        2
DSD GROCERY                  1
HOUSEHOLD CHEMICALS/SUPP     1
MEAT - FRESH & FROZEN        1
PAINT AND ACCESSORIES       19
PETS AND SUPPLIES            4
Name: ScanCount, dtype: int64

### 2-0-1. Train TripType

In [5]:
samsub = pd.read_csv('./data/sample_submission.csv')

In [6]:
samsub.tail()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
95669,191338,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95670,191339,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95671,191340,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95672,191341,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95673,191348,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_TT_dummy_df = pd.get_dummies(train['TripType'], prefix='TripType_')

In [8]:
train_TT_dummy_df.tail()

,TripType__3,TripType__4,TripType__5,TripType__6,TripType__7,TripType__8,TripType__9,TripType__12,TripType__14,TripType__15,...,TripType__36,TripType__37,TripType__38,TripType__39,TripType__40,TripType__41,TripType__42,TripType__43,TripType__44,TripType__999
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647052,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-1-1. Train Weekday

In [9]:
# before
train['Weekday'].value_counts()

Sunday       133975
Saturday     122096
Friday        96247
Monday        83130
Tuesday       72529
Wednesday     71115
Thursday      67962
Name: Weekday, dtype: int64

In [10]:
WD_columns = train['Weekday'].value_counts().index
WD_columns

Index(['Sunday', 'Saturday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'],
      dtype='object')

In [11]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

train['Weekday'] = weekday_preprocessing(train)
train['Weekday'].value_counts()

1    133975
2    122096
3     96247
4     83130
5     72529
6     71115
7     67962
Name: Weekday, dtype: int64

In [12]:
train_WD_dummy_df = pd.get_dummies(train['Weekday'])
train_WD_dummy_df.columns = WD_columns

In [13]:
train_WD_dummy_df.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday
647049,1,0,0,0,0,0,0
647050,1,0,0,0,0,0,0
647051,1,0,0,0,0,0,0
647052,1,0,0,0,0,0,0
647053,1,0,0,0,0,0,0


### 2-1-2. Test Weekday

In [14]:
# before
test['Weekday'].value_counts()

Sunday       133233
Saturday     123974
Friday        94764
Monday        85174
Tuesday       73742
Wednesday     72481
Thursday      70278
Name: Weekday, dtype: int64

In [15]:
WD_columns = test['Weekday'].value_counts().index
WD_columns

Index(['Sunday', 'Saturday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'],
      dtype='object')

In [16]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

test['Weekday'] = weekday_preprocessing(test)
test['Weekday'].value_counts()

1    133233
2    123974
3     94764
4     85174
5     73742
6     72481
7     70278
Name: Weekday, dtype: int64

In [17]:
test_WD_dummy_df = pd.get_dummies(test['Weekday'])
test_WD_dummy_df.columns = WD_columns

In [18]:
test_WD_dummy_df.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday
653641,1,0,0,0,0,0,0
653642,1,0,0,0,0,0,0
653643,1,0,0,0,0,0,0
653644,1,0,0,0,0,0,0
653645,1,0,0,0,0,0,0


---

### 2-2. Upc Skip

---

### 2-3-1. Train DepartmentDescription

In [19]:
print(len(train['DepartmentDescription'].sort_values().unique()))
train['DepartmentDescription'].sort_values().unique()

67


array(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY',
       'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES',
       'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES',
       'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD',
       'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY',
       'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES',
       'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14',
       'GROCERY DRY GOODS', 'HARDWARE', 'HEALTH AND BEAUTY AIDS',
       'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS',
       'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS',
       'IMPULSE MERCHANDISE', 'INFANT APPAREL',
       'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES',
       'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS',
       'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN',
       'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES',
       'OPTICAL - FRAMES', 'OPTICAL - LENSES', '

In [20]:
train_DD_dummy_df = pd.get_dummies(train['DepartmentDescription'])
train_DD_dummy_df.tail()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# train_DD_dummy_df_columns=[]
# for idx in preprocessing.LabelEncoder().fit_transform(train_DD_dummy_df.columns):
#     train_DD_dummy_df_columns.append('DD_{}'.format(idx))
# train_DD_dummy_df.columns = train_DD_dummy_df_columns
# train_DD_dummy_df.tail()

### 2-3-2. Test DepartmentDescription

In [22]:
print(len(test['DepartmentDescription'].sort_values().unique()))
test['DepartmentDescription'].sort_values().unique()

66


array(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY',
       'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES',
       'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES',
       'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD',
       'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY',
       'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES',
       'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14',
       'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT',
       'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP',
       'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL',
       'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES',
       'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS',
       'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN',
       'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES',
       'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS',
       'PAINT

'HEALTH AND BEAUTY AIDS' DD가 test에는 없다.!!!

In [24]:
test_DD_dummy_df = pd.get_dummies(test['DepartmentDescription'])
test_DD_temp_dummy_df = pd.concat([test_DD_dummy_df.loc[:,test_DD_dummy_df.columns[:26]], 
                                   pd.DataFrame([0]*len(test), columns=['HEALTH AND BEAUTY AIDS'])], axis=1)
test_DD_dummy_df = pd.concat([test_DD_temp_dummy_df,
                              test_DD_dummy_df.loc[:,test_DD_dummy_df.columns[26:]]],axis=1)
test_DD_dummy_df.tail()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
653641,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# test_DD_dummy_df_columns=[]
# for idx in preprocessing.LabelEncoder().fit_transform(test_DD_dummy_df.columns):
#     test_DD_dummy_df_columns.append('DD_{}'.format(idx))
# test_DD_dummy_df.columns = test_DD_dummy_df_columns
# test_DD_dummy_df.tail()

### 2-4-1. Train FinelineNumber

In [26]:
len(train['FinelineNumber'].sort_values().unique())

5195

In [28]:
%%time
# in Test, not in Train
test_unique_fln = test['FinelineNumber'].sort_values().unique()
train_unique_fln = train['FinelineNumber'].sort_values().unique()

in_test_not_in_train = [i for i in test_unique_fln if i not in train_unique_fln]

CPU times: user 210 ms, sys: 12.9 ms, total: 223 ms
Wall time: 221 ms


In [29]:
len(in_test_not_in_train)

158

In [30]:
train_FLN_dummy_df = pd.get_dummies(train['FinelineNumber'], prefix='FLN_')
train_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN__9964.0,FLN__9966.0,FLN__9967.0,FLN__9970.0,FLN__9971.0,FLN__9974.0,FLN__9975.0,FLN__9991.0,FLN__9997.0,FLN__9998.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
FLN_zero_series = pd.Series([0]*len(train))
train_FLN_zero_dummy_df = pd.concat([FLN_zero_series, FLN_zero_series],axis=1)
for _ in tqdm(range(6)):
    train_FLN_zero_dummy_df = pd.concat([train_FLN_zero_dummy_df, train_FLN_zero_dummy_df],axis=1)

for _ in tqdm(range(30)):
    train_FLN_zero_dummy_df = pd.concat([train_FLN_zero_dummy_df, FLN_zero_series],axis=1)

100%|██████████| 6/6 [00:01<00:00,  4.15it/s]


In [34]:
train_FLN_zero_dummy_df.columns = ['FLN_'+str(col) for col in in_test_not_in_train]
train_FLN_zero_dummy_df.tail()

,FLN_92.0,FLN_186.0,FLN_187.0,FLN_480.0,FLN_489.0,FLN_546.0,FLN_573.0,FLN_646.0,FLN_678.0,FLN_743.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
%%time 
train_all_FLN_dummy_df = pd.concat([train_FLN_dummy_df, train_FLN_zero_dummy_df], axis=1)

CPU times: user 54.8 s, sys: 4.35 s, total: 59.1 s
Wall time: 59.5 s


In [36]:
train_all_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-4-2. Test FinelineNumber

In [37]:
len(test['FinelineNumber'].sort_values().unique())

5203

In [38]:
test_FLN_dummy_df = pd.get_dummies(test['FinelineNumber'], prefix='FLN_')
test_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN__9967.0,FLN__9969.0,FLN__9970.0,FLN__9971.0,FLN__9974.0,FLN__9975.0,FLN__9991.0,FLN__9997.0,FLN__9998.0,FLN__9999.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
%%time
# in Train, not in Test
in_train_not_in_test = \
[i for i in train_unique_fln if i not in test_unique_fln]

CPU times: user 36.3 ms, sys: 1.89 ms, total: 38.2 ms
Wall time: 36.4 ms


In [40]:
len(in_train_not_in_test)

150

In [41]:
FLN_zero_series = pd.Series([0]*len(test))
test_FLN_zero_dummy_df = pd.concat([FLN_zero_series, FLN_zero_series],axis=1)
for _ in tqdm(range(6)):
    test_FLN_zero_dummy_df = pd.concat([test_FLN_zero_dummy_df, test_FLN_zero_dummy_df],axis=1)

for _ in tqdm(range(22)):
    test_FLN_zero_dummy_df = pd.concat([test_FLN_zero_dummy_df, FLN_zero_series],axis=1)

100%|██████████| 22/22 [00:30<00:00,  1.39s/it]


In [42]:
test_FLN_zero_dummy_df.columns = ['FLN_'+str(col) for col in in_train_not_in_test]
test_FLN_zero_dummy_df.tail()

,FLN_45.0,FLN_238.0,FLN_245.0,FLN_332.0,FLN_373.0,FLN_524.0,FLN_551.0,FLN_567.0,FLN_593.0,FLN_830.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
%%time 
test_all_FLN_dummy_df = pd.concat([test_FLN_dummy_df, test_FLN_zero_dummy_df], axis=1)

CPU times: user 59.7 s, sys: 6.18 s, total: 1min 5s
Wall time: 1min 6s


In [44]:
test_all_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-5-1. Train ScanCount

In [45]:
%%time
# in Train, not in Test
train_unique_sc = train['ScanCount'].sort_values().unique()
test_unique_sc = test['ScanCount'].sort_values().unique()

SC_in_test_not_in_train = \
[i for i in test_unique_sc if i not in train_unique_sc]

CPU times: user 74.1 ms, sys: 13 ms, total: 87.1 ms
Wall time: 85.6 ms


In [46]:
SC_in_test_not_in_train

[-21, -14, -11, -8, 26, 27, 28, 29, 35, 36, 40, 41, 42, 45, 55]

In [47]:
train_SC_dummy_df = pd.get_dummies(train['ScanCount'],prefix='SC_')

In [48]:
train_SC_dummy_df.tail()

,SC__-12,SC__-10,SC__-9,SC__-7,SC__-6,SC__-5,SC__-4,SC__-3,SC__-2,SC__-1,...,SC__20,SC__22,SC__23,SC__24,SC__25,SC__30,SC__31,SC__46,SC__51,SC__71
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
SC_zero_series = pd.Series([0]*len(train))
train_SC_zero_dummy_df = pd.concat([SC_zero_series, SC_zero_series],axis=1)
for _ in range(13):
    train_SC_zero_dummy_df = pd.concat([train_SC_zero_dummy_df, SC_zero_series],axis=1)

In [50]:
train_SC_zero_dummy_df.columns = ['SC_'+str(col) for col in SC_in_test_not_in_train]
train_SC_zero_dummy_df.tail()

,SC_-21,SC_-14,SC_-11,SC_-8,SC_26,SC_27,SC_28,SC_29,SC_35,SC_36,SC_40,SC_41,SC_42,SC_45,SC_55
647049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
%%time 
train_all_SC_dummy_df = pd.concat([train_SC_dummy_df, train_SC_zero_dummy_df], axis=1)

CPU times: user 132 ms, sys: 67.2 ms, total: 199 ms
Wall time: 199 ms


In [52]:
train_all_SC_dummy_df.tail()

,SC__-12,SC__-10,SC__-9,SC__-7,SC__-6,SC__-5,SC__-4,SC__-3,SC__-2,SC__-1,...,SC_27,SC_28,SC_29,SC_35,SC_36,SC_40,SC_41,SC_42,SC_45,SC_55
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-5-2. Test ScanCount

In [53]:
%%time
# in Train, not in Test
SC_in_train_not_in_test = \
[i for i in train_unique_sc if i not in test_unique_sc]

CPU times: user 265 µs, sys: 1e+03 ns, total: 266 µs
Wall time: 271 µs


In [54]:
SC_in_train_not_in_test

[-12, 25, 30, 51, 71]

In [55]:
test_SC_dummy_df = pd.get_dummies(test['ScanCount'],prefix='SC_')

In [56]:
test_SC_dummy_df.tail()

,SC__-21,SC__-14,SC__-11,SC__-10,SC__-9,SC__-8,SC__-7,SC__-6,SC__-5,SC__-4,...,SC__29,SC__31,SC__35,SC__36,SC__40,SC__41,SC__42,SC__45,SC__46,SC__55
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
SC_zero_series = pd.Series([0]*len(test))
test_SC_zero_dummy_df = pd.concat([SC_zero_series, SC_zero_series],axis=1)
for _ in range(3):
    test_SC_zero_dummy_df = pd.concat([test_SC_zero_dummy_df, SC_zero_series],axis=1)

In [58]:
test_SC_zero_dummy_df.columns = ['SC_'+str(col) for col in SC_in_train_not_in_test]
test_SC_zero_dummy_df.tail()

,SC_-12,SC_25,SC_30,SC_51,SC_71
653641,0,0,0,0,0
653642,0,0,0,0,0
653643,0,0,0,0,0
653644,0,0,0,0,0
653645,0,0,0,0,0


In [59]:
%%time 
test_all_SC_dummy_df = pd.concat([test_SC_dummy_df, test_SC_zero_dummy_df], axis=1)

CPU times: user 162 ms, sys: 39.7 ms, total: 201 ms
Wall time: 200 ms


In [61]:
test_all_SC_dummy_df.tail()

,SC__-21,SC__-14,SC__-11,SC__-10,SC__-9,SC__-8,SC__-7,SC__-6,SC__-5,SC__-4,...,SC__41,SC__42,SC__45,SC__46,SC__55,SC_-12,SC_25,SC_30,SC_51,SC_71
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

## Preprocessing Step3 - Concat VisitNumber & all Dummy DFs

### Train

- VisitNumber & Weekday

In [62]:
%time train_concated_df = pd.concat([train['VisitNumber'],train_WD_dummy_df],axis=1)

CPU times: user 7.25 ms, sys: 1.33 ms, total: 8.58 ms
Wall time: 7.68 ms


- append ScanCount

In [63]:
%time train_concated_df = pd.concat([train_concated_df,train_all_SC_dummy_df],axis=1)

CPU times: user 42.1 ms, sys: 31.2 ms, total: 73.3 ms
Wall time: 71.6 ms


- append DepartmentDescription

In [64]:
%time train_concated_df = pd.concat([train_concated_df,train_DD_dummy_df],axis=1)

CPU times: user 410 ms, sys: 104 ms, total: 514 ms
Wall time: 513 ms


- append FinelineNumber

In [65]:
%time train_concated_df = pd.concat([train_concated_df,train_all_FLN_dummy_df],axis=1)

CPU times: user 2.23 s, sys: 3.92 s, total: 6.15 s
Wall time: 6.9 s


In [66]:
train_concated_df.shape

(647054, 5482)

In [67]:
train_concated_df.tail()

,VisitNumber,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday,SC__-12,SC__-10,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,191346,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,191346,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,191346,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,191347,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,191347,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 파생변수 만들기

In [75]:
a = train_concated_df[train_concated_df['VisitNumber']==8]

In [86]:
b = a.drop(columns=[c for c in a.columns if  'FLN_' in c])

In [87]:
b.tail()

,VisitNumber,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday,SC__-12,SC__-10,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
21,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
b.groupby('')

- groupby VisitNumber

In [ ]:
# %%time
# final_train = train_concated_df.groupby('VisitNumber').sum()

In [ ]:
# final_train.shape

In [ ]:
# %time train_groupbyed_TripType = train.groupby('VisitNumber')['TripType'].agg(lambda x: scipy.stats.mode(x)[0])

In [ ]:
# len(train_groupbyed_TripType)

In [ ]:
# %time final_train = pd.concat([train_groupbyed_TripType,final_train],axis=1)

In [ ]:
# final_train.tail()

In [ ]:
# %time final_train.to_csv('./data/final_train_without_upc.csv')

In [ ]:
# %time pickle.dump(final_train, open("./data/final_train.pkl", "wb"))

### test

- VisitNumber & Weekday

In [68]:
%time test_concated_df = pd.concat([test['VisitNumber'],test_WD_dummy_df],axis=1)

CPU times: user 8.77 ms, sys: 11.5 ms, total: 20.3 ms
Wall time: 18.7 ms


- append ScanCount

In [69]:
%time test_concated_df = pd.concat([test_concated_df,test_all_SC_dummy_df],axis=1)

CPU times: user 37.7 ms, sys: 48.3 ms, total: 86 ms
Wall time: 84.6 ms


- append DepartmentDescription

In [70]:
%time test_concated_df = pd.concat([test_concated_df,test_DD_dummy_df],axis=1)

CPU times: user 151 ms, sys: 166 ms, total: 317 ms
Wall time: 322 ms


- append FinelineNumber

In [71]:
%time test_concated_df = pd.concat([test_concated_df,test_all_FLN_dummy_df],axis=1)

CPU times: user 2.26 s, sys: 4.16 s, total: 6.42 s
Wall time: 7.21 s


In [72]:
test_concated_df.shape

(653646, 5482)

- groupby VisitNumber

In [ ]:
# %time final_test = test_concated_df.groupby('VisitNumber').sum()

In [ ]:
# %time pickle.dump(final_test, open("./data/final_test.pkl", "wb"))

In [ ]:
final_test.shape

In [ ]:
final_test.tail()

In [ ]:
# %time final_test.to_csv('./data/final_test_without_upc.csv')

In [ ]:
myslack.send_slack('work finish')

---